In [37]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Read data into a dataframe and 
# df = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = df.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]
# densityPlot = sns.kdeplot(counts, bw_adjust=3)
# yLabels = densityPlot.get_yticks()
# densityPlot.set_yticklabels('{:,.0%}'.format(y) for y in yLabels)
# plt.xlabel("Number of reviews")
# plt.ylabel("Percentage of users")
# plt.savefig(fname="densityPlot")
# plt.show()

In [38]:
# import pandas as pd

# # Read ratings into a dataframe and narrow down reviewers to desired review count
# ratings = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = ratings.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]

# # Create and apply mask that removes all other reviewers.
# mask = ratings['reviewerID'].isin(counts.index)
# ratings = ratings[mask]

# # Read metadata into dataframe and define asins that are used by making a set
# metadata = pd.read_csv("metadata.csv")
# used_asins = set(ratings["asin"])

# # Create and apply mask that removes unreferenced items
# mask = metadata["asin"].isin(used_asins)
# metadata = metadata[mask]
# metadata.to_pickle("metadata.pickle")
# ratings.to_pickle("ratings.pickle")


In [39]:
# import pandas as pd
# import numpy as np
# from scipy.sparse import csr_matrix, save_npz
# ratings : pd.DataFrame = pd.read_pickle("ratings.pickle")
# metadata : pd.DataFrame = pd.read_pickle("metadata.pickle")
# ratings = ratings[["reviewerID", "asin", "overall"]]

# # Series of each unique value of reviewerID and asin 
# reviewerIDs = ratings['reviewerID'].unique()
# asins = ratings["asin"].unique()
# # Dictionary that maps reviewers to an index
# reviewerMap = {reviewerID: i for i, reviewerID in enumerate(reviewerIDs)}

# # Dictionary that maps asins to an index
# asinMap = {asin: i for i, asin in enumerate(asins)}

# # Initialize the utility matrix with zeros
# utilityMatrixArray = np.zeros((len(reviewerIDs), len(asins)), "int8")

# # Fill in the utility matrix with data from the records array
# for row in ratings.itertuples():
#     reviewerIndex = reviewerMap[row.reviewerID]
#     asinIndex = asinMap[row.asin]
#     utilityMatrixArray[reviewerIndex, asinIndex] = row.overall
# utilityMatrix = csr_matrix(utilityMatrixArray)
# save_npz("utilityMatrix.npz", utilityMatrix)

In [40]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors

# Get all data needed
ratings : pd.DataFrame = pd.read_pickle("ratings.pickle")
metadata : pd.DataFrame = pd.read_pickle("metadata.pickle")
utilityMatrix = load_npz("utilityMatrix.npz")

# Series of each unique value of reviewerID and asin 
reviewerIDs = ratings['reviewerID'].unique()
asins = ratings["asin"].unique()

# Dictionaries that maps reviewers and asins to indexes
reviewerMap = {reviewerID: i for i, reviewerID in enumerate(reviewerIDs)}
asinMap = {asin: i for i, asin in enumerate(asins)}

# Creation of test 
hoodies = metadata[metadata['category'].apply(lambda c: "Hoodie" in c)]
hoodieAsins = hoodies["asin"]
testAsins = hoodieAsins[5:9]
testIndices = testAsins.index.to_list()
print(testIndices)
utilityMatrix = utilityMatrix

# create a NearestNeighbors model with cosine similarity
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

# fit the model to the item vectors
model.fit(utilityMatrix.T)

# get the nearest neighbors of test items
_, indices = model.kneighbors(utilityMatrix.T[testIndices], n_neighbors=20)

# iterate through the test items
for i, idx in enumerate(testIndices):
    # get the indices of the nearest neighbors
    neighbor_indices = indices[i][1:]
    # get the asin of the test item
    test_asin = metadata.loc[idx]['asin']
    print(f"Recommendations for {test_asin}:")
    # iterate through the nearest neighbors and print their asins
    for neighbor_idx in neighbor_indices:
        neighbor_asin = metadata.iloc[neighbor_idx]['asin']
        print(f"- {neighbor_asin}")


[536, 543, 589, 861]
141755
Recommendations for B0008IVI3M:
- B0184ICVTW
- B01DE8P02W
- B0008IVCPG
- B015FPFGNK
- B00Z04C3E2
- B00TXZN4RY
- B00SFIDN4Y
- B00CADKY0Q
- B00K256GK6
- B00UZIX0OU
- B0031B9UWO
- B00RWAZOHU
- B007Z4YHRG
- B00ID4R6OI
- B0008IVCOW
- B00BD3ZC1K
- B0008IVCQ0
- B00CAI8EZ8
- B000A09ZXS
Recommendations for B0008MF4QK:
- B015J1IGCI
- B0127Z84IA
- B01ECNJM1I
- B01EDI9U2S
- B00YFSD33E
- B008F9VT0S
- B010AEZH6W
- B005IB0OHA
- B00NS8S4YK
- B00VCQ09KW
- B00PUR27HC
- B0082O268A
- B00FAVYDRK
- B00K5K0MKI
- B007BRX8G8
- B00FMMTUIO
- B00KBRXMBG
- B00DEYLKQW
- B00FAVLBF2
Recommendations for B0009G9QWI:
- B014GDW1Z8
- B014GBLJSK
- B015YO0W5O
- B00OHZZL6C
- B00E41VEE2
- B00LIQ81JC
- B00FZ0MVA2
- B00FRR02J0
- B00OYWYJIY
- B00C3PQDV0
- B01C63YA7C
- B019WZQ0AW
- B0042UR0XE
- B00QKSGJ6A
- B0082E6GHW
- B00U10F0A4
- B0052SUE8I
- B00RCOTGXE
- B00S9XB982
Recommendations for B000B6A5HG:
- B00QMLO3MM
- B00K58RQZO
- B00S4QICSE
- B004HKSDO4
- B00K7I4BG4
- B00LU6VGN8
- B00DQ3OS38
- B00FDPUB4W